In [ ]:
library(tidyverse)

In [ ]:
filetype = "ratio"

input_folder = "final_pipeline_NEW"
input_phase <- "phaseII"

In [ ]:
paramspace = read.csv("../data/metadata/paramspace_cfDNA_phaseII_missing_removed.csv")


paramspace  <- paramspace %>% rename(patient_id = pt_id)

In [ ]:
head(paramspace)

In [ ]:


phase2ptex <- read.csv("../phase2_exclude3.csv", header=FALSE)
phase2ptex <- as.character(unlist(c(phase2ptex[1,])))

paramspace <- paramspace %>% filter(!patient_id %in% phase2ptex)



In [ ]:
SAMPLES = paramspace["patient_id"]
FOLDERS = paramspace["cfDNA_folder"]

In [ ]:
nrow(paramspace)

In [ ]:
c2i_res <- read.csv("../data/metadata/clin_data/c2i_res.csv")
head(c2i_res)
c2i_res$Sample.ID <- as.character(c2i_res$Sample.ID)
c2i_res$C2.Test <- as.character(c2i_res$C2.Test)
c2i_res <- c2i_res %>% mutate(patient_id = paste0("C0", Subject.ID), sampleID = substr(Sample.ID,1,nchar(Sample.ID)-6)) %>%
    select(patient_id, sampleID, C2.Test, Tumor.Fraction)
c2i_res$patient_id <- as.character(c2i_res$patient_id) 

c2i_res <- c2i_res %>% filter(patient_id %in% unique(paramspace$patient_id))
head(c2i_res)
str(c2i_res)
dim(c2i_res)

In [ ]:
#paramspace["big_ratio"] <- NA
#paramspace["big_ratio_CI_lower"] <- NA
#paramspace["big_ratio_CI_upper"] <- NA

paramspace["small_ratio"] <- NA
paramspace["small_ratio_CI_lower"] <- NA
paramspace["small_ratio_CI_upper"] <- NA

for (row in 1:nrow(paramspace)){
    pt = paramspace[row, "patient_id"]
    #print(pt)
    fd = paramspace[row, "cfDNA_folder"]
    #big_r = read.csv(paste("../data/", pt, "/" , fd, "/", input_folder, "/", "big_", filetype, ".csv", sep = ""))
    #small_r = read.csv(paste("../data/phase_II_patients/", pt, "/", fd, "/", input_folder, "/", "small_", filetype, ".csv", sep = ""))
    small_r = read.csv(paste("../data/phase_II_patients/", pt, "/", fd, "/", input_folder, "/", "small_", filetype, ".csv", sep = ""))
    #paramspace[row, "big_ratio"] = big_r$ratio
    #paramspace[row, "big_ratio_CI_lower"] = big_r$lower_CI
    #paramspace[row, "big_ratio_CI_upper"] = big_r$upper_CI
    
    paramspace[row, "small_ratio"] = small_r$ratio
    paramspace[row, "small_ratio_CI_lower"] = small_r$lower_CI
    paramspace[row, "small_ratio_CI_upper"] = small_r$upper_CI
}

In [ ]:
res <- paramspace

In [ ]:
res <- res %>% 
  mutate(tmp_chunks = stringr::str_split(cfDNA_folder, stringr::fixed("_"),  n = 3)) %>%
  mutate(sampleID = map_chr(tmp_chunks, 1),
         sub_value = map_chr(tmp_chunks, 2), 
         sub_val = map_chr(tmp_chunks, 3)) %>%
  select(-c(tmp_chunks, sub_value, sub_val))

head(res)
dim(res)

In [ ]:
sample_timepoint <- read.csv("../data/metadata/clin_data/sample_timepoint_formatted.csv")

In [ ]:
head(sample_timepoint)
sample_timepoint_days <- sample_timepoint %>% filter(phase == input_phase)
sample_timepoint_days <- sample_timepoint_days %>% select(sampleID, sample_timepoint_days_since_OP)
sample_timepoint_days %>% filter(is.na(sample_timepoint_days_since_OP))
dim(sample_timepoint_days)
head(sample_timepoint_days)

In [ ]:
res$sampleID <- as.character(res$sampleID)
res <- left_join(res, sample_timepoint_days, by = c("sampleID"))
head(res)
dim(res)

In [ ]:
clinical_data <- read.csv("../data/metadata/clin_data/clinical_data_formatted.csv")
clinical_data_relapse <- clinical_data %>% select(patient_id, time_to_relapse_days, adjuvant_chemo_start_days, adjuvant_chemo_end_days)
head(clinical_data_relapse)

In [ ]:
res <-left_join(res, clinical_data_relapse, by="patient_id")
head(res)
dim(res)

In [ ]:
res_plot <- res %>% select(patient_id, cfDNA_folder, sampleID, small_ratio, sample_timepoint_days_since_OP, time_to_relapse_days)
dim(res_plot)
str(res_plot)

In [ ]:
normalized_res_intervention <- data.frame(matrix(ncol = 8, nrow = 0))
colnames(normalized_res_intervention) <- c("patient_id", "cfDNA_folder", "sampleID", "small_ratio", "sample_timepoint_days_since_OP", "time_to_relapse_days","small_ratio_normalized_preop", "small_ratio_normalized_postop")

In [ ]:
patients = as.character(unique(res_plot$patient_id))
normalized_res_intervention <- NULL

for (i in patients){
    res_pt <- res_plot %>% filter(patient_id == i)
    
    first_sample <- min(res_pt$sample_timepoint_days_since_OP)
    without_first_sample <- res_pt %>% filter(sample_timepoint_days_since_OP != first_sample)
    post_op_sample <- min(without_first_sample$sample_timepoint_days_since_OP)
    
    pre_val_small_pre <- res_pt$small_ratio[which(res_pt$sample_timepoint_days_since_OP == first_sample)]
    pre_val_small_post <- res_pt$small_ratio[which(res_pt$sample_timepoint_days_since_OP == post_op_sample)]
    res_pt <- res_pt %>% mutate(small_ratio_normalized_preop = small_ratio/pre_val_small_pre, small_ratio_normalized_postop = small_ratio/pre_val_small_post)
    normalized_res_intervention <- rbind(normalized_res_intervention, res_pt)   
}

normalized_res_intervention <- normalized_res_intervention %>% mutate(small_ratio_normalized_postop_log = log10(small_ratio_normalized_postop), relapse_status_pred = ifelse(small_ratio_normalized_postop_log > 0.5, "D", "ND"))
head(normalized_res_intervention)

In [ ]:
relapse <- read.csv("../data/metadata/clin_data//clinical_data_formatted.csv")
relapse <- relapse %>% filter(C2i_phase == input_phase) %>% select(patient_id, is_relapse..0.no.1.yes., time_to_relapse_days)
relapse_label <- relapse %>% mutate(Relapse_label = ifelse(is_relapse..0.no.1.yes. == 0, "No_relapse", "Relapse")) %>% select(patient_id, Relapse_label)
head(relapse_label)

In [ ]:
dim(normalized_res_intervention)
normalized_res_intervention <- left_join(normalized_res_intervention, relapse_label, by = "patient_id")
head(normalized_res_intervention)
dim(normalized_res_intervention)
str(normalized_res_intervention)

# Imaging based

In [ ]:
length(unique(normalized_res_intervention$patient_id))

In [ ]:
normalized_res_intervention$Relapse_label <- as.character(normalized_res_intervention$Relapse_label)
normalized_res_intervention <- normalized_res_intervention %>% filter(!patient_id %in% phase2ptex)
truth <- normalized_res_intervention %>% filter(Relapse_label == "Relapse")
truth$patient_id <- as.character(truth$patient_id)
unique(truth$patient_id)

In [ ]:
normalized_res_intervention$Relapse_label <- as.character(normalized_res_intervention$Relapse_label)
truth_nor <- normalized_res_intervention %>% filter(Relapse_label == "No_relapse")
truth_nor$patient_id <- as.character(truth_nor$patient_id)
unique(truth_nor$patient_id)

In [ ]:
normalized_res_intervention %>% filter(Relapse_label == "Relapse") %>% filter(!patient_id %in% phase2ptex) %>%  group_by(patient_id) %>%
    summarize(time_to_relapse_imaging = min(time_to_relapse_days))

# K-mer based

In [ ]:
pre_ops <- normalized_res_intervention %>% group_by(patient_id) %>% summarize(pre_op = min(sample_timepoint_days_since_OP))

In [ ]:
normalized_res_intervention <- normalized_res_intervention %>% mutate(small_ratio_normalized_postop_log = log10(small_ratio_normalized_postop))
dim(normalized_res_intervention)

In [ ]:
normalized_res_intervention_rm_preop <- normalized_res_intervention %>% filter(!sample_timepoint_days_since_OP %in% pre_ops$pre_op)
dim(normalized_res_intervention_rm_preop)

In [ ]:
detected_kmers <- normalized_res_intervention_rm_preop %>% group_by(patient_id) %>% summarize(max_ratio_log = max(small_ratio_normalized_postop_log)) %>% filter(max_ratio_log > 0.5)
detected_kmers$patient_id <- as.character(detected_kmers$patient_id)
true_detections <- unique(detected_kmers$patient_id)[(unique(detected_kmers$patient_id) %in% unique(truth$patient_id))]
true_detections
false_detections <- unique(detected_kmers$patient_id)[!(unique(detected_kmers$patient_id) %in% unique(truth$patient_id))]
false_detections


In [ ]:
normalized_res_intervention_rm_preop %>% filter(Relapse_label == "Relapse") %>% filter(small_ratio_normalized_postop_log > 0.5) %>% group_by(patient_id) %>%
    summarize(sample_timepoint_days_since_OP_relapse = min(sample_timepoint_days_since_OP))

# C2i

In [ ]:
c2i_res <- left_join(c2i_res, sample_timepoint_days, by = c("sampleID")) 
c2i_res <- c2i_res %>% filter(!is.na(sample_timepoint_days_since_OP))
head(c2i_res)
dim(c2i_res)

In [ ]:
c2i_res_rm_preop <- c2i_res %>% filter(!sample_timepoint_days_since_OP %in% pre_ops$pre_op)
dim(c2i_res_rm_preop)

In [ ]:
c2i_detected <- c2i_res_rm_preop %>% filter(C2.Test == "D") %>% filter(!patient_id %in% phase2ptex)
true_detections <- unique(c2i_detected$patient_id)[(unique(c2i_detected$patient_id) %in% unique(truth$patient_id))]
true_detections
false_detections <- unique(c2i_detected$patient_id)[!(unique(c2i_detected$patient_id) %in% unique(truth$patient_id))]
false_detections

In [ ]:
c2i_res_rm_preop %>% filter(patient_id %in% true_detections) %>% 
    filter(C2.Test == "D") %>% 
    group_by(patient_id) %>% 
    summarize(min_detected =  min(sample_timepoint_days_since_OP))